In [73]:
print("Hello")

# Sustainable mobile phone display


Hello


In [74]:
import pandas as pd
import os


# Directory where your CSV files are located
folder_path = './summarized_patents/'


# List to hold data from each CSV file
data_frames = []


# Loop through all the CSV files in the directory
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Read the CSV file and append it to the list of data frames
        df = pd.read_csv(file_path)
        data_frames.append(df)


# Concatenate all the data frames into one
merged_df = pd.concat(data_frames, ignore_index=True)


# Save the merged data frame to a new CSV file
output_file_path = './test_summarization_loop/merged_file.csv'
merged_df.to_csv(output_file_path, index=False)


print(f'Merged CSV saved to {output_file_path}')


Merged CSV saved to ./test_summarization_loop/merged_file.csv


In [27]:
import pandas as pd
data_patent = pd.read_csv('./test_summarization_loop/merged_file.csv')

In [29]:
rows = len(data_patent.axes[0])
cols = len(data_patent.axes[1])
print(rows, cols)

3720 11


In [30]:
patent_data = data_patent[["PatentNumber","ApplicationReference","Date","OrganizationName","SummaryofPatent"]]

In [31]:
rows = len(patent_data.axes[0])
cols = len(patent_data.axes[1])
print(rows, cols)

3720 5


In [32]:
patent_data.to_csv('./test_summarization_loop/patents_summarized_abstract_clean.csv', index=False)

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [2]:
loader = CSVLoader(file_path='./test_summarization_loop/patents_summarized_abstract_clean.csv')

documents = loader.load()

In [3]:
print(type(documents))

<class 'list'>


In [5]:
documents[0].page_content

'PatentNumber: 11925278\nApplicationReference: utility\nDate: 2024-03-12\nOrganizationName: SAMSUNG ELECTRONICS CO., LTD.\nSummaryofPatent: Summary:\nThe display apparatus aims to provide a flexible and adjustable mounting system for displays, allowing for rotation in multiple directions and slidable movement in the leftward and rightward directions. The connector includes rotatable supports and movable supports, which enable the mounting member to be securely attached to the fixing member while allowing for adjustment. The invention improves upon existing mounting systems by providing a more flexible and adjustable solution, but may be limited by the weight and size of the display. The energy efficiency of the invention is not explicitly mentioned in the provided text.'

In [4]:
len(documents)

3720

In [44]:
#Embedding model load

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

model_name = "BAAI/bge-large-en-v1.5"
# model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    # ,
    # query_instruction="为这个句子生成表示以用于检索相关文章："
)


emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())

[codecarbon WARNING @ 00:04:00] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 00:04:00] [setup] RAM Tracking...
[codecarbon INFO @ 00:04:00] [setup] GPU Tracking...
[codecarbon INFO @ 00:04:00] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 00:04:00] [setup] CPU Tracking...
[codecarbon WARNING @ 00:04:00] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1717801440.4403791


[codecarbon WARNING @ 00:04:01] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 00:04:01] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 00:04:01] >>> Tracker's metadata:
[codecarbon INFO @ 00:04:01]   Platform system: Linux-5.4.0-182-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 00:04:01]   Python version: 3.12.1
[codecarbon INFO @ 00:04:01]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 00:04:01]   Available RAM : 503.339 GB
[codecarbon INFO @ 00:04:01]   CPU count: 112
[codecarbon INFO @ 00:04:01]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 00:04:01]   GPU count: 2
[codecarbon INFO @ 00:04:01]   GPU model: 2 x NVIDIA A40
[codecarbon INFO @ 00:04:07] Energy consumed for RAM : 0.000116 kWh. RAM Power : 188.75217247009277 W
[codecarbon INFO @ 00:04:07] Energy consumed for all GPUs : 0.000096 kWh. Total GPU Power : 155.277552940

Emissions for this code execution is:  6.225456186088755e-05
End of code time:  1717801447.1427157


In [8]:
print(model)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-large-en-v1.5' cache_folder=None model_kwargs={} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: ' embed_instruction=''


In [9]:
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations

In [42]:
 # Clear out the existing database directory if it exists
if os.path.exists("./db"):
    shutil.rmtree("./db")


In [10]:
#Chroma DB saving files and embeddings

from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

chroma_db = Chroma.from_documents(
    documents, model, persist_directory="./db"
)

chroma_db.persist()

emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())


[codecarbon WARNING @ 17:56:45] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 17:56:45] [setup] RAM Tracking...
[codecarbon INFO @ 17:56:45] [setup] GPU Tracking...
[codecarbon INFO @ 17:56:45] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:56:45] [setup] CPU Tracking...
[codecarbon WARNING @ 17:56:45] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1716224205.5088346


[codecarbon WARNING @ 17:56:46] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 17:56:46] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 17:56:46] >>> Tracker's metadata:
[codecarbon INFO @ 17:56:46]   Platform system: Linux-5.4.0-174-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 17:56:46]   Python version: 3.12.1
[codecarbon INFO @ 17:56:46]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 17:56:46]   Available RAM : 503.339 GB
[codecarbon INFO @ 17:56:46]   CPU count: 112
[codecarbon INFO @ 17:56:46]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 17:56:46]   GPU count: 2
[codecarbon INFO @ 17:56:46]   GPU model: 2 x NVIDIA A40
[codecarbon INFO @ 17:57:04] Energy consumed for RAM : 0.000787 kWh. RAM Power : 188.75217247009277 W
[codecarbon INFO @ 17:57:04] Energy consumed for all GPUs : 0.001460 kWh. Total GPU Power : 349.876985650

Emissions for this code execution is:  0.0014553289802961846
End of code time:  1716224245.5109322


In [43]:
import torch

# del model

torch.cuda.empty_cache()
torch.cuda.mem_get_info()

NameError: name 'model' is not defined

In [45]:
from langchain.vectorstores import Chroma
vectordb = Chroma(persist_directory="./db", embedding_function= model)


In [46]:
retriever = vectordb.as_retriever(search_type = "similarity_score_threshold",search_kwargs = {"k": 10, "score_threshold":0.50})
# retriever = vectordb.as_retriever(search_type = "mmr",search_kwargs = {"k": 5, "lambda":0.90})
print(retriever.search_type, retriever.search_kwargs)

similarity_score_threshold {'k': 10, 'score_threshold': 0.5}


In [70]:
retirve_result = retriever.get_relevant_documents("Energy Efficient and Sustainable Touch Screen Display")

In [64]:
len(retirve_result)

10

In [33]:
# retirve_result[0]

In [28]:
# import re

# pattern = re.compile(r"SummaryofPatent: Summary:\n(.*)", re.DOTALL)

# match = pattern.search(retirve_result[0].page_content)

# if match:
#     print(match.group(1).strip())

The invention aims to improve the efficiency of wireless charging by designing a spiral-shaped antenna with a unique structure. The antenna consists of two conductive patterns, one for current flowing inside and one for current flowing outside, which are formed of multiple layers. The design helps to reduce energy loss and improve charging efficiency. However, the presence of multiple layers may increase manufacturing complexity. The invention does not provide details on energy efficiency, but the unique structure is expected to improve charging efficiency.


In [21]:
# retirve_result[0]

Document(page_content='PatentNumber: 11955597\nApplicationReference: utility\nDate: 2024-04-09\nOrganizationName: Samsung Electronics Co., Ltd.\nSummaryofPatent: Summary:\nThe research aims to develop a solid electrolyte for lithium-ion batteries that improves energy efficiency and reduces degradation. The proposed electrolyte is based on oxides of hafnium, titanium, zirconium, or a combination thereof, and shows improved stability and conductivity compared to traditional electrolytes. However, the study mentions that the electrolyte may degrade under high-temperature conditions, and further research is needed to overcome this limitation. The energy efficiency of the proposed electrolyte is not explicitly mentioned in the text.', metadata={'row': 2228, 'source': './test_summarization_loop/patents_summarized_abstract_clean.csv'})

In [36]:
# filtered_results = [doc for score, doc in retirve_result['SummaryofPatent'] if score >= "0.7"]
# print(filtered_results)

In [12]:
# type(retirve_result)

list

In [34]:
# import re

# for item in retirve_result:
#     pattern = re.compile(r"SummaryofPatent: Summary:\n(.*)", re.DOTALL)

#     match = pattern.search(item.page_content)

#     if match:
#         print(match.group(1).strip())

In [71]:
import re

# Assuming the Document class has an attribute 'page_content' that stores the content
class Document:
    def __init__(self, page_content):
        self.page_content = page_content

for item in retirve_result:
    # Regular expression to capture Patent Number and Summary
    pattern = re.compile(r"PatentNumber: (\d+)\n.*SummaryofPatent: Summary:\n(.*)", re.DOTALL)

    match = pattern.search(item.page_content)  # Use dot notation to access the page_content attribute

    if match:
        patent_number = match.group(1).strip()
        summary = match.group(2).strip()
        print("Patent Number:", patent_number)
        print("Summary:", summary)


Patent Number: 11910672
Summary: The invention aims to improve the energy efficiency of touch screens by overlapping touch lines with power supply lines. The non-display area of the substrate includes touch electrodes, touch lines connected to the electrodes, and first and second power supply lines with different voltage levels. The first power supply line overlaps the second power supply line, and at least one of the touch lines overlaps one of the power supply lines. This design can reduce the energy consumption of the touch screen by minimizing the distance between the touch electrodes and the power supply lines. However, the invention does not provide details on the specific energy efficiency improvements or the potential drawbacks of the design.
Patent Number: 11875732
Summary: The research aims to improve the energy efficiency of display panels by developing a screen saver controller that generates screen saver data based on load and life data of panel blocks. The controller adju

In [72]:
from IPython.display import display, HTML

style = """
<style>
    .research-summary {
        border: 1px solid #ddd;
        border-radius: 8px;
        padding: 15px;
        margin: 10px 0;
        background-color: #f9f9f9;
        color: black;
    }
    .research-summary h3 {
        color: #336;
    }
    .research-summary p {
        text-indent: 20px;
    }
</style>
"""

import re

# Assuming the Document class has an attribute 'page_content' that stores the content
class Document:
    def __init__(self, page_content):
        self.page_content = page_content

for item in retirve_result:
    # Regular expression to capture Patent Number and Summary
    pattern = re.compile(r"PatentNumber: (\d+)\n.*SummaryofPatent: Summary:\n(.*)", re.DOTALL)

    match = pattern.search(item.page_content)  # Use dot notation to access the page_content attribute

    if match:
        contents = ""
        patent_number = match.group(1).strip()
        summary = match.group(2).strip()
        # print("Patent Number:", patent_number)
        # print("Summary:", summary)
        contents += f"""
            <div class="research-summary">
                <h3>Patent Number:{patent_number}</h3>
                <p>{summary}</p>
            </div>
            """
        display(HTML(style + contents))


